In [1]:
import numpy as np
from numpy import pi, exp, log10
from numpy.fft import fft, fftshift, fftfreq, ifft
from bokeh.plotting import figure, show, output_notebook
import bokeh.palettes as pl
import panel as pn
pn.extension()

import utils

import matplotlib.pyplot as plt
import scipy

def labels(p, x='', y='', t=''):
    p.xaxis.axis_label=x
    p.yaxis.axis_label=y
    p.title=t

def fontsize(p):
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'
    p.xaxis.axis_label_text_font_size = '12pt'
    p.yaxis.axis_label_text_font_size = '12pt'
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.title.text_font_size = '12pt'
    p.toolbar.logo = None

def ticks(p):
    p.xaxis.ticker.num_minor_ticks = 10
    p.yaxis.ticker.num_minor_ticks = 3

def bplot(x):
    p = figure(width=800, height=400)
    p.line(np.arange(len(x)), x, line_width=2)
    p.toolbar.logo = None
    show(p)

def bplot2(n, x):
    p = figure(width=800, height=400)
    p.line(n, x)
    p.toolbar.logo = None
    show(p)

In [2]:
fs, x1 = scipy.io.wavfile.read('freebird.wav')
fs, x1.shape

gain = 100

In [3]:
# cut down waveform size to two minutes centered around the start of the freebird solo
start = 5*60 + 41 + 30
end = 7*60 + 41 - 30
x = x1[start*fs:end*fs]/gain

This creates the $\mathbf{B}$ matrix which consists of 1s and 0s and sums the 2049 FFT bins into 34 frequency bands

In [4]:
def b_matrix():
    B = np.zeros((34, 2049))
    r = np.round(np.linspace(0, 33, 2049)).astype(int)
    for i in range(34):
        B[i, r == i] = 1
    return B

The cross spectrogram is defined as:
$$\rho(\mathbf{X}, \mathbf{Y}) = \mathbf{B}(\mathbf{X} \times \mathbf{Y}^*)$$
where $\times$ denotes element-wise multiplication, and $*$ denotes element-wise complex conjugation.

In [5]:
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import hann

M = 4096 # frame size
overlap = 0.75 # hann window overlap (used to calculate hop size)
hop = int(M*(1-overlap)) # hop size

window = hann(M)

stft = ShortTimeFFT(win=window, hop=hop, fs=fs)

In [6]:
l = x[:, 0]
r = x[:, 1]
s = (l + r) / 2 # mono signal
N = len(l)

P_IID, P_IC = utils.encode(stft, l, r)
X = 10**(np.abs(P_IID)/10)

In [7]:
# test the inverse
# l1 = inverse_spectrogram(stft, L).astype(np.int16)
# r1 = inverse_spectrogram(stft, R).astype(np.int16)
# scipy.io.wavfile.write('freebird_stft.wav', fs, np.array([l1, r1]).T);

In [8]:
from bokeh import palettes as pl
from bokeh.models import LogColorMapper, ColorBar

m = LogColorMapper(palette=pl.Inferno256, low=X.min(), high=X.max())

p = figure(width=1500, height=700, title='Spectrogram', x_axis_label='Time (s)', y_axis_label='Frequency (kHz)')
p.min_border=0

p.image(image=[X], x=0, y=0, dw=X.shape[1]*hop/fs, dh=fs/2/1e3, color_mapper=m)
p.add_layout(ColorBar(color_mapper=m), 'right')

show(p)

In [9]:
y_mixed = utils.decode(stft, s, P_IID, P_IC)*gain
scipy.io.wavfile.write('freebird_decoded.wav', fs, y_mixed.T.astype(np.int16))